In [1]:
'''
Code to perform evaluation on 2DSyn_BC dataset
'''

import os
os.chdir("..")

import numpy as np
from copy import deepcopy
import itertools
import torch
import tifffile
import random

from scipy.ndimage import uniform_filter

import data
from data.sampler.synBC2D_sampler import SynBC2DSampler
from options.options import Options
from util.mesh_handler import *
from util.util import *
from metric_funcs import *

from models.biospade_model import BioSPADEModel
import models.networks as networks

import matplotlib.pyplot as plt

n_patches = 5 #200 # Number of patches to test
n_instances = 5 #25 # Number of instances per patch to test
name = 'Syn2D_BC' # Name of experiments

# Generate Real Data
def generate(vox, powers, frames, blur=True, std=None, n_instances=1, rand_noise=True):
    if len(powers) != len(frames):
        raise ValueError('Powers and Frames are not equal in length')
    
    if blur:
        vox = blur_vox(vox, org_opt.sigmas)

    bin_mat = None
    p = org_opt.p_blobs
    real = np.zeros([n_instances, len(powers), *vox.shape[-2:]])
    for j in range(len(powers)):
        if not rand_noise:
            bin_mat = np.random.choice([0.0, 1.0], size=vox.shape, p=[1-p,p])
        for i in range(n_instances):
            real[i, j] = sampler.generate(vox, powers[j], frames[j], std_scalar=std, bin_mat=bin_mat)
    return real

# Generate noise for each layer of G
def generate_noise(sz):
    xy_sz = sz[-2:]
    
    noise = []
    for i in range(num_up_layers+1):
        x_sz = xy_sz[0]//(2**i)
        y_sz = xy_sz[1]//(2**i)

        noise += [torch.randn([*sz[:2],x_sz, y_sz]).cuda()]
    return noise

# Generate fake images
def forward(model, vox, powers, frames, n_instances=1, rand_noise=True, is_cycle=False):
    if len(powers) != len(frames):
        raise ValueError('Powers and Frames are not equal in length')

    fake = torch.zeros([n_instances, len(powers),*vox.shape[-2:]])
    fake_mu = torch.zeros_like(fake)
    noise = generate_noise([len(powers),*vox.shape[-3:]])
    for i in range(n_instances):
        data = {'mesh_semantics': len(powers)*[vox],
                'power': powers,
                'frames': frames,
                'z_pos': len(powers)*[0]}
        data = tensorize_dict(data)
        if not rand_noise:
            data['noise'] = noise
            
        if is_cycle:
            model.set_input(data)
            fake_ = model.forward(True)
            fake[i] = torch.relu(fake_[:,0])
        else:
            fake_, fake_mu_ = model(data,'inference')
            fake[i], fake_mu[i] = fake_[:,0,0], fake_mu_[:,0,0]
    return fake, fake_mu

In [2]:
# Set Parameters

tag = 'synBC2D'
org_opt = Options('options/test_options.yaml', tag)
org_opt.train_mode = 'GAN'
org_opt.initialize()

org_opt.dataset_mode = tag
org_opt.name = name
org_opt.how_many_patches = n_patches

org_opt.samples_per_instance = 1

num_up_layers = networks.generator.compute_latent_vector_size(org_opt)

dataloader, dataset = data.create_dataloader(org_opt, 'all')
sampler = SynBC2DSampler(org_opt)
files = dataset.mesh_paths

style_combs = np.asarray(list(itertools.product(org_opt.powers, org_opt.frames)))

stack dataset [SynBC2DDataset] of size 41 was created
mesh dataset [SynBC2DDataset] of size 41 was created


In [3]:
# Generate testing data

test_input = np.zeros([org_opt.how_many_patches, 1, *org_opt.crop_xy_sz])
real_input = np.zeros([org_opt.how_many_patches, 1, *org_opt.crop_xy_sz])
test_gt = np.zeros([org_opt.how_many_patches, *org_opt.crop_xy_sz])
real_patches = np.zeros([org_opt.how_many_patches, n_instances, len(style_combs), *org_opt.crop_xy_sz])

n_samples = 0
while n_samples<org_opt.how_many_patches:
    batch = next(iter(dataloader))
    input = batch['mesh_semantics'].view(-1,1,*org_opt.crop_xy_sz).numpy()
    real_input_ = batch['real_semantics'].view(-1,1,*org_opt.crop_xy_sz).numpy()
    
    n_input = min([len(input), org_opt.how_many_patches-n_samples])
    test_input[n_samples:n_samples+n_input] = input[:n_input]
    real_input[n_samples:n_samples+n_input] = real_input_[:n_input]
    
    n_samples += n_input

for i, input in enumerate(test_input):
    real_patches[i] = generate(input, style_combs[:,0], style_combs[:,1].astype(int), n_instances=n_instances, rand_noise=True)
    test_gt[i] = uniform_filter(input, 3)>1e-2

In [4]:
# Perform Evaluation

out_sz = (org_opt.number_of_experiments, len(style_combs))
losses = {'NMSE': np.zeros(out_sz),
          'PSNR': np.zeros(out_sz),
          'JSD': np.zeros(out_sz),
          'GLCM': np.zeros(out_sz),
          'LBP': np.zeros(out_sz)}

real = real_patches.swapaxes(0,2)
avg_real = real.mean(1)
blur_real = blur_all(real)
text_real = blur_real[:,0]

real_PSNR = calc_PSNR(real, axis=(1,2,3,4)) # Compute real PSNR
real_LBP = calc_LBP(text_real[:,:,None]) # Compute real LBP
real_GLCM = calc_GLCM(text_real[:,:,None]) # Compute real GLCM

for exp in range(0, org_opt.number_of_experiments):
    fake_patches = np.zeros([org_opt.how_many_patches, n_instances, len(style_combs), *org_opt.crop_xy_sz])

    # Get experiment details
    opt = deepcopy(org_opt)
    exp_str = ''
    if not org_opt.run_all:
        exp_str = '(exp'+str(exp)+')'
        opt.set_experiment(exp)
    opt.name = org_opt.name+exp_str

    # Load model
    print('Evaluating:', opt.name)
    model = BioSPADEModel(opt)
    model.eval()

    # Generate data
    for i, input in enumerate(test_input):
        fake_patches[i],_  = forward(model, input[:,None], 
                                     style_combs[:,0], style_combs[:,1], 
                                     n_instances=n_instances)
        
    fake = fake_patches.swapaxes(0,2)
    avg_fake = fake.mean(1)
    text_fake = blur_all(fake)[:,0]
    
    fake_LBP = calc_LBP(text_fake[:,:,None])
    fake_GLCM = calc_GLCM(text_fake[:,:,None])
    
    losses['NMSE'][exp] = MSE(avg_fake*test_gt, avg_real*test_gt, normalize=True) # Compute NMSE
    losses['GLCM'][exp] = MSE(fake_GLCM, real_GLCM,axis=(1,2,3)) # Compute MSE of GLCM
    losses['LBP'][exp] = mult_JSDs(fake_LBP, real_LBP) # Compute JSD of LBP
    losses['PSNR'][exp] = (calc_PSNR(fake, axis=(1,2,3,4))-real_PSNR)**2 # Compute MSE of PSNR
    losses['JSD'][exp] = compare_hist(fake, real) # Compute JSD of pixel values

Evaluating: Syn2D_BC
Network [SPADEGenerator] was created. Total number of parameters: 8.3 million. To see the architecture, do print(network).


In [5]:
# Print Results

keys = ['NMSE', 'LBP', 'GLCM', 'PSNR', 'JSD']
scale = {'NMSE': 1e1, 'GLCM': 1e8, 'LBP': 1e3, 'PSNR':1e0, 'JSD': 1e2}

ln = ''
for key in keys:
    ln += key+' & '
print(ln)

for i_loss in range(org_opt.number_of_experiments):
    ln = 'EXP '+str(i_loss+1)+': ' # Set to '' if you want to be able to copy and past table into excel
    for key in keys:
        val = losses[key][i_loss].mean()
        ln += '%0.3f & ' % (val*scale[key])
    print(ln)

NMSE & LBP & GLCM & PSNR & JSD & 
EXP 1: 1.594 & 3.687 & 0.061 & 0.003 & 0.017 & 
